# Mediapipe포즈 추출

In [5]:
import os
import cv2
import mediapipe as mp
import pandas as pd

In [6]:
# Mediapipe Pose 모델 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

In [7]:
# 이미지 디렉토리 및 파일 리스트
image_dir = r"C:\Users\admin\Desktop\ZB\ZB_DL_proj\image_dataset"  # 이미지가 저장된 디렉토리
image_list = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

In [8]:
# 키포인트를 CSV로 저장
output_data = []

In [9]:
for img_file in image_list:
    img_path = os.path.join(image_dir, img_file)
    image = cv2.imread(img_path)
    if image is None:
        print(f"이미지를 읽을 수 없습니다: {img_file}")
        continue

    # Mediapipe로 포즈 추출
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_image)

    if results.pose_landmarks:
        keypoints = {}
        for idx in range(11, 17):  # Upper body
            keypoints[f"x_{idx}"] = results.pose_landmarks.landmark[idx].x
            keypoints[f"y_{idx}"] = results.pose_landmarks.landmark[idx].y
            keypoints[f"z_{idx}"] = results.pose_landmarks.landmark[idx].z
        for idx in range(23, 29):  # Lower body
            keypoints[f"x_{idx}"] = results.pose_landmarks.landmark[idx].x
            keypoints[f"y_{idx}"] = results.pose_landmarks.landmark[idx].y
            keypoints[f"z_{idx}"] = results.pose_landmarks.landmark[idx].z

        keypoints["image_name"] = img_file
        output_data.append(keypoints)

# 결과를 CSV 파일로 저장
df = pd.DataFrame(output_data)
df.to_csv("keypoints.csv", index=False)

print("키포인트 추출 완료 및 CSV 저장!")

키포인트 추출 완료 및 CSV 저장!


# CSV에 라벨 추가

In [15]:
import pandas as pd

# Mediapipe에서 생성된 키포인트 CSV 파일 로드
df = pd.read_csv("keypoints.csv")

# 파일명에서 라벨 생성
def assign_label(filename):
    if filename.startswith("fall"):
        return 1
    elif filename.startswith("normal"):
        return 0
    else:
        return "unknown"

df["label"] = df["image_name"].apply(assign_label)

df = df.drop('image_name', axis = 1)
# 결과 확인
print(df.head())

# 새로운 CSV 파일 저장
df.to_csv("keypoints_with_labels.csv", index=False)
print("라벨링이 완료된 CSV 파일이 저장되었습니다.")

       x_11      y_11      z_11      x_12      y_12      z_12      x_13  \
0  0.563721  0.489814  0.108493  0.520806  0.499862  0.037136  0.582124   
1  0.594460  0.730538 -0.091645  0.608566  0.648975 -0.228065  0.566723   
2  0.796953  0.754577 -0.096929  0.780289  0.681341  0.117584  0.784828   
3  0.556454  0.517310  0.189455  0.518578  0.506342  0.166118  0.574975   
4  0.798352  0.754972 -0.103873  0.779680  0.681517  0.109419  0.784379   

       y_13      z_13      x_14  ...      x_26      y_26      z_26      x_27  \
0  0.531343  0.095130  0.527933  ...  0.564713  0.545647 -0.197328  0.606547   
1  0.683567 -0.093195  0.563450  ...  0.532665  0.579429 -0.066236  0.541710   
2  0.799267 -0.256963  0.735494  ...  0.610379  0.705216  0.060092  0.565586   
3  0.544570  0.181897  0.501672  ...  0.548308  0.633264 -0.204786  0.603548   
4  0.798960 -0.258889  0.735832  ...  0.607777  0.707006  0.063541  0.563799   

       y_27      z_27      x_28      y_28      z_28  label  
0  0.73

# 정규화

In [16]:
from sklearn.preprocessing import MinMaxScaler

# 키포인트 컬럼만 선택
keypoint_columns = [col for col in df.columns if col.startswith(("x_", "y_", "z_"))]

# Min-Max 정규화 수행
scaler = MinMaxScaler()
df[keypoint_columns] = scaler.fit_transform(df[keypoint_columns])

# 정규화 결과 저장
df.to_csv("normalized_keypoints.csv", index=False)
print("정규화가 완료된 CSV 파일이 저장되었습니다.")

정규화가 완료된 CSV 파일이 저장되었습니다.


# 모델 학습

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D
import numpy as np


In [18]:
# CSV 파일 로드
file_path = r'C:\Users\admin\Desktop\ZB\ZB_DL_proj\project_code\normalized_keypoints.csv'  # 파일 경로 수정
df = pd.read_csv(file_path)

In [19]:
# 라벨 분리
X = df.drop("label", axis=1).values  # Numpy 배열로 변환
y = df["label"].values  # Numpy 배열로 변환

# 학습/검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 차원 확장 (CNN 입력에 맞게 변환)
X_train_cnn = X_train[..., np.newaxis]  # (samples, features, 1)
X_test_cnn = X_test[..., np.newaxis]  # (samples, features, 1)

In [24]:
# CNN 모델 설계
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train_cnn, y_train, validation_data=(X_test_cnn, y_test), epochs=100, batch_size=32)

# 모델 평가
loss, accuracy = model.evaluate(X_test_cnn, y_test)
print(f"test loss: {loss:.4f}, test accuracy: {accuracy:.4f}")


Epoch 1/100


c:\Users\admin\anaconda3\envs\yolo_study\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5267 - loss: 0.6949 - val_accuracy: 0.8652 - val_loss: 0.6573
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7261 - loss: 0.6375 - val_accuracy: 0.8708 - val_loss: 0.5332
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7891 - loss: 0.5363 - val_accuracy: 0.8483 - val_loss: 0.4344
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8152 - loss: 0.4651 - val_accuracy: 0.9045 - val_loss: 0.2953
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8292 - loss: 0.3768 - val_accuracy: 0.9270 - val_loss: 0.2164
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8687 - loss: 0.2895 - val_accuracy: 0.9270 - val_loss: 0.1914
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8857 - loss: 0.2794 - val_accuracy: 0.9326 - val_loss: 0.1822
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8997 - loss: 0.2335 - val_accuracy: 0.9326 - val_loss: 0.1

In [25]:
from sklearn.metrics import confusion_matrix, classification_report

# 모델 예측
y_pred = model.predict(X_test_cnn)
y_pred = (y_pred > 0.5).astype(int)  # 0.5를 기준으로 양성/음성 분류

# 혼동 행렬 계산
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

# 민감도(Sensitivity)와 특이도(Specificity) 계산
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

print(f"혼동 행렬:\n{cm}")
print(f"민감도(Sensitivity, Recall): {sensitivity:.4f}")
print(f"특이도(Specificity): {specificity:.4f}")

# 추가적인 분류 보고서
report = classification_report(y_test, y_pred, target_names=["Normal", "Abnormal"])
print(f"\n분류 보고서:\n{report}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
혼동 행렬:
[[87  1]
 [ 6 84]]
민감도(Sensitivity, Recall): 0.9333
특이도(Specificity): 0.9886

분류 보고서:
              precision    recall  f1-score   support

      Normal       0.94      0.99      0.96        88
    Abnormal       0.99      0.93      0.96        90

    accuracy                           0.96       178
   macro avg       0.96      0.96      0.96       178
weighted avg       0.96      0.96      0.96       178

